In [ ]:
'''
Apache Spark is an open-source cluster-computing framework, built around speed, ease of use, and 
streaming analytics whereas Python is a general-purpose, high-level programming language. 
It provides a wide range of libraries and is majorly used for Machine Learning and Real-Time Streaming Analytics.

In other words, it is a Python API for Spark that lets you harness the simplicity of Python and

the power of Apache Spark in order to tame Big Data. 
'''

In [3]:
import os
import pandas as pd
import numpy as np

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator

import seaborn as sns
import matplotlib.pyplot as plt




In [9]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 400)

from matplotlib import rcParams
sns.set(context='notebook', style='whitegrid', rc={'figure.figsize': (18,4)})
rcParams['figure.figsize'] = 18,4

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [11]:
#setting random seed for notebook reproducabilty
rnd_seed =23
np.random.seed=rnd_seed
np.random.set_state=rnd_seed


In [21]:
#Creating spark session
SparkSession.builder.master("local[2]").appName("Linear-Regression-California-Housing").getOrCreate()

Exception: Java gateway process exited before sending its port number